In [1]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


In [2]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [3]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [5]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
         validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 131us/step - loss: 11.5015 - acc: 0.1130 - val_loss: 11.5025 - val_acc: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 50us/step - loss: 11.4882 - acc: 0.1110 - val_loss: 11.4982 - val_acc: 0.1300
Epoch 3/10
1000/1000 [==============================] - 0s 64us/step - loss: 11.4781 - acc: 0.1360 - val_loss: 11.4969 - val_acc: 0.0900
Epoch 4/10
1000/1000 [==============================] - 0s 57us/step - loss: 11.4720 - acc: 0.1370 - val_loss: 11.4958 - val_acc: 0.1200
Epoch 5/10
1000/1000 [==============================] - 0s 59us/step - loss: 11.4656 - acc: 0.1510 - val_loss: 11.4969 - val_acc: 0.1300
Epoch 6/10
1000/1000 [==============================] - 0s 68us/step - loss: 11.4593 - acc: 0.1500 - val_loss: 11.5047 - val_acc: 0.1000
Epoch 7/10
1000/1000 [==============================] - 0s 63us/step - loss: 11.4541 - acc: 0.1650 - val_loss: 11.5036 - val_acc: 

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
         validation_data=val_dataset, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 9ms/step - loss: 11.3857 - acc: 0.2167 - val_loss: 11.5210 - val_acc: 0.1250
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3651 - acc: 0.2448 - val_loss: 11.4988 - val_acc: 0.1667
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3436 - acc: 0.2521 - val_loss: 11.6250 - val_acc: 0.0938
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3356 - acc: 0.2615 - val_loss: 11.6711 - val_acc: 0.0938
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3064 - acc: 0.2594 - val_loss: 11.5355 - val_acc: 0.1250
Epoch 6/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3442 - acc: 0.2667 - val_loss: 11.5036 - val_acc: 0.0729
Epoch 7/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3215 - acc: 0.2729 - val_loss: 11.6249 - val_acc: 0.1042
Epoch 8/10
30/30 [==============================] - 0s 2ms/step - loss: 11.3

In [8]:
model.evaluate(data, labels, batch_size=32)

model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 3ms/step


[11.333526357014973, 0.25]

In [10]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


In [11]:
inputs = tf.keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

In [13]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 327us/step - loss: 11.6687 - acc: 0.1100
Epoch 2/5
1000/1000 [==============================] - 0s 49us/step - loss: 11.5505 - acc: 0.1160
Epoch 3/5
1000/1000 [==============================] - 0s 58us/step - loss: 11.5092 - acc: 0.1030
Epoch 4/5
1000/1000 [==============================] - 0s 57us/step - loss: 11.4959 - acc: 0.1220
Epoch 5/5
1000/1000 [==============================] - 0s 67us/step - loss: 11.4864 - acc: 0.1330


In [14]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.dense_1 = layers.Dense(32, activation='relu')
        self.dense_2 = layers.Dense(num_classes, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [15]:
model = MyModel(num_classes=10)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 366us/step - loss: 11.7041 - acc: 0.0950
Epoch 2/5
1000/1000 [==============================] - 0s 70us/step - loss: 11.6052 - acc: 0.0980
Epoch 3/5
1000/1000 [==============================] - 0s 57us/step - loss: 11.5319 - acc: 0.1050
Epoch 4/5
1000/1000 [==============================] - 0s 65us/step - loss: 11.5138 - acc: 0.0960
Epoch 5/5
1000/1000 [==============================] - 0s 66us/step - loss: 11.5059 - acc: 0.0980


In [16]:
class MyLayer(layers.Layer):
    
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel',
                                     shape=shape,
                                     initializer='uniform',
                                     trainable=True)
        super(MyLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [17]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 341us/step - loss: 11.5159 - acc: 0.0970
Epoch 2/5
1000/1000 [==============================] - 0s 62us/step - loss: 11.4981 - acc: 0.0880
Epoch 3/5
1000/1000 [==============================] - 0s 50us/step - loss: 11.4920 - acc: 0.0910
Epoch 4/5
1000/1000 [==============================] - 0s 50us/step - loss: 11.4896 - acc: 0.1000
Epoch 5/5
1000/1000 [==============================] - 0s 64us/step - loss: 11.4887 - acc: 0.1020


In [18]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
         validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 185us/step - loss: 11.4861 - acc: 0.1120 - val_loss: 11.4890 - val_acc: 0.1300
Epoch 2/5
1000/1000 [==============================] - 0s 81us/step - loss: 11.4839 - acc: 0.1100 - val_loss: 11.4904 - val_acc: 0.0600
Epoch 3/5
1000/1000 [==============================] - 0s 70us/step - loss: 11.4824 - acc: 0.1190 - val_loss: 11.4880 - val_acc: 0.0900
Epoch 4/5
1000/1000 [==============================] - 0s 80us/step - loss: 11.4813 - acc: 0.1140 - val_loss: 11.4924 - val_acc: 0.0800
Epoch 5/5
1000/1000 [==============================] - 0s 76us/step - loss: 11.4796 - acc: 0.1220 - val_loss: 11.4952 - val_acc: 0.0900


In [19]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')])
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [20]:
model.save_weights('./weights/my_model')

model.load_weights('./weights/my_model')

In [21]:
model.save_weights('my_model.h5', save_format='h5')

model.load_weights('my_model.h5')

In [22]:
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "Dense", "config": {"name": "dense_8", "trainable": true, "dtype": null, "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_9", "trainable": true, "dtype": null, "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "kera

In [23]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense

In [26]:
fresh_model = tf.keras.models.model_from_json(json_string)

In [27]:
yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: null
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
      kernel_regularizer: null
      name: dense_8
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: null
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
      kernel_regularizer: null
      name: dense_9
      tra

In [28]:
fresh_model = tf.keras.models.model_from_yaml(yaml_string)

In [30]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax', input_shape=(32,)),
    layers.Dense(10, activation='softmax')])
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

model.save('my_model.h5')
model = tf.keras.models.load_model('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 443us/step - loss: 11.5060 - acc: 0.0990
Epoch 2/5
1000/1000 [==============================] - 0s 66us/step - loss: 11.4937 - acc: 0.1010
Epoch 3/5
1000/1000 [==============================] - 0s 72us/step - loss: 11.4893 - acc: 0.1040
Epoch 4/5
1000/1000 [==============================] - 0s 67us/step - loss: 11.4876 - acc: 0.1040
Epoch 5/5
1000/1000 [==============================] - 0s 70us/step - loss: 11.4869 - acc: 0.1120


In [31]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax'),
    layers.Dense(10, activation='softmax')])
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/f5/yns8fq1s48g03ncqddxz0rmm0000gn/T/tmph4o7oloa', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x15d1a6cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
